# Estudo da composição das previsões do rating final

O objetivo é realizar previsões para toda base gerando um rating unindo o contribuinte e posteriormente o igr gerado na regressão

In [1]:
import os
import dotenv
import boto3
from io import BytesIO
import pandas as pd
from datetime import date
import zipfile
from sklearn.preprocessing import StandardScaler, RobustScaler
import pickle

import warnings

warnings.filterwarnings("ignore")

print("Processo iniciado")

# Define diretorios
rootPath = os.getcwd()
dataPath = os.path.join(rootPath, 'data')
modelsPath = os.path.join(rootPath, 'models')
env = os.path.join(rootPath, '.env')
dotenv.load_dotenv(dotenv_path=env)

Processo iniciado


True

In [2]:
s3 = boto3.resource(
    service_name='s3',
    region_name='us-east-1',
    aws_access_key_id=os.getenv("AWS_ACESS_KEY"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACESS_KEY")
)

In [5]:
print("Carregando dados para previsões")

# Prepara os dados para realizar as previsões

zip_file = os.path.join(dataPath, 'base_treino.zip')
z = zipfile.ZipFile(zip_file)


def ler_bases_exportadas(nome_arquivo):
    z.extract(nome_arquivo)
    df = pd.read_csv(nome_arquivo, sep=',')
    os.remove(nome_arquivo)
    return df

base_conjunta = ler_bases_exportadas('imovel_mercantil.csv')


Carregando dados para previsões


In [59]:
xx = base_conjunta[base_conjunta['vlr_pago']> 0]
xx[['cda', 'tipo_divida', 'valor_tot', 'vlr_pago']]

,cda,tipo_divida,valor_tot,vlr_pago
7,000040eda866e3d19ceed47545e621dd1,imovel,10779.87,5438.40
16,00006f113b5e15229ceed47545e621dd1,imovel,12302.26,3522.19
21,0000905d980177f36b3a8e268c80aedf1,imovel,528.49,528.49
22,0000a43dae672b166b3a8e268c80aedf1,imovel,3786.64,1692.36
23,0000a43dae672b166b3a8e268c80aedf2,mercantil,1928.97,1928.97
...,...,...,...,...
2414881,ffff9425053e623a6b3a8e268c80aedf1,imovel,111112.73,111112.73
2414882,ffff9cef7028d1f47c8fa7a67092eaab1,imovel,119580.77,38010.00
2414891,ffffd79d8aa612ac9fcacff532cf50c41,imovel,26104.42,26104.42
2414892,ffffd79d8aa612ac9fcacff532cf50c42,mercantil,2504.85,265.15


In [6]:
base_pred = pd.read_csv(r'C:\Users\Consultor\Documents\bases_pesquisa\verificacao\imovel_mercantil_pred.csv')

In [58]:
ww = base_pred[base_pred['vlr_pago'] >0]
ww[['cda', 'tipo_divida', 'valor_tot', 'vlr_pago']]

,cda,tipo_divida,valor_tot,vlr_pago
48424,0ca6a542e26c8a4518565e26094f06ee1,imovel,1294.99,1219.92
56487,0ec3ddb649aa29101,imovel,6102.97,2560.59
102094,1a9f7f4800abe5876b3a8e268c80aedf1,imovel,5983.34,2840.85
198194,33b0b07752a115bcb5eae1a923f0f8701,imovel,321.72,206.75
212080,3750dbd6719eb43618565e26094f06ee1,imovel,2624.00,1743.06
288506,4b0e64d1047d11de2b46f673bf0c61251,imovel,4050.44,3970.15
289486,4b4bb36d34e33e849ceed47545e621dd1,imovel,1468.15,933.71
315828,521ceda796df75899ceed47545e621ddE,mercantil,990.07,238.35
358446,5d34cbca2cadb7309fcacff532cf50c42,mercantil,290.27,185.60
467854,79858779dfcb24c79989bae6f4af91ee1,imovel,149.18,118.74


In [18]:
soma_treino = base_conjunta[base_conjunta['da_aberto'] == 1]['valor_tot'].sum()
soma_treino



16035665438.840002

 da_aberto        cda        valor_tot       vlr_pago
 1              larissa        3000             0
 0              larissa        300             300

resultado - previsao
 1              larissa        3300            300

In [32]:
duplicatas = base_pred[base_pred['cda'].duplicated(keep=False)]
duplicatas[['cda']]

,cda


In [63]:
aux = base_conjunta[base_conjunta['da_aberto'] == 1]
aux

,cda,tipo_divida,id_contribuinte,atividade_principal,situacao,tipo_tributo,vlr_pago,valor_tot,vlr_tributo,vlr_taxa,...,edificacao,cpf_cnpj_existe,protesto,ajuizamento,refis,deb_totais,deb_pagos,idade_divida,quantidade_reparcelamento,da_aberto
0,00000123c3d4731c6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,APARTAMENTO,ATIVO,IPTU,0.00,2048.73,762.08,1286.65,...,1,1,0,0,0,40.0,0.0,1.0,0,1
1,00000123c3d4731c6b3a8e268c80aedf2,mercantil,fc4b99b807fbed41,ATIVIDADES DE TELEATENDIMENTO,SUSPENSO,ISS,0.00,2515.85,0.00,2515.85,...,0,1,0,0,0,3.0,0.0,1.0,0,1
3,000014e359592e62d8a3e5cebc255ca6E,mercantil,6dbe14da38a31dc1,Comercio varejista especializado de equipament...,ATIVO,ISS,0.00,847.02,847.02,0.00,...,0,1,0,0,0,1.0,0.0,0.0,0,1
5,0000331f601a73e52b46f673bf0c61252,mercantil,e8424494daac9641,"COM VAR DE MERC EM GERAL, COM PREDOM DE PROD A...",INAPTO,ISS,0.00,5385.81,0.00,5385.81,...,0,1,0,1,0,12.0,0.0,11.0,0,1
6,00003d46e618da886b3a8e268c80aedf1,imovel,d8b23eda9800b9e3,CASA,EM PROCESSO DE BAIXA,IPTU,0.00,6434.96,3409.70,3025.26,...,1,0,0,1,0,60.0,0.0,12.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2414888,ffffca43e897bbf0d8a3e5cebc255ca61,imovel,cad70f5b8701af03,CASA,ATIVO,IPTU,0.00,5082.97,2240.39,2842.58,...,1,0,0,1,0,40.0,0.0,11.0,0,1
2414889,ffffca43e897bbf0d8a3e5cebc255ca62,mercantil,e750ab1d88feb1ba,ATIVIDADES DE ASSOCIAÇÕES DE DEFESA DE DIREITO...,ATIVO,ISS,0.00,7634.36,0.00,7634.36,...,0,1,0,1,0,8.0,0.0,11.0,0,1
2414892,ffffd79d8aa612ac9fcacff532cf50c42,mercantil,cc5afb41f6fbc367,CONSULTORIA EM TECNOLOGIA DA INFORMAÇÃO,ATIVO,ISS,265.15,2504.85,2504.85,0.00,...,0,1,0,0,0,4.0,2.0,1.0,1,1
2414895,ffffdc78aa7a90e26b3a8e268c80aedf1,imovel,9b512f210b460120,APARTAMENTO,ATIVO,IPTU,0.00,9390.43,5113.61,4276.82,...,1,1,0,1,0,30.0,0.0,25.0,0,1


In [42]:
# Cdas que estão na query vw_da_treino e não estao na query vw_da_predicao
cdas_exclusivas = base_conjunta[~base_conjunta ['cda'].isin(base_pred['cda'])]
cdas_exclusivas

,cda,tipo_divida,id_contribuinte,atividade_principal,situacao,tipo_tributo,vlr_pago,valor_tot,vlr_tributo,vlr_taxa,...,edificacao,cpf_cnpj_existe,protesto,ajuizamento,refis,deb_totais,deb_pagos,idade_divida,quantidade_reparcelamento,da_aberto
2,00000b44c5ba1e669ceed47545e621dd2,mercantil,96e8e553de69d7a4,COMERCIO VAREJISTA DE ARTIGOS DE ARMARINHO,INAPTO,ISS,0.00,1278.97,0.00,1278.97,...,0,1,0,0,0,4.0,0.0,21.0,0,0
4,0000331f601a73e52b46f673bf0c61251,imovel,870c08c252b25ad1,APARTAMENTO,ATIVO,IPTU,0.00,344.12,147.20,196.92,...,1,1,0,0,0,6.0,0.0,11.0,0,0
7,000040eda866e3d19ceed47545e621dd1,imovel,9f3bac8718dac1fa,LOJA,ATIVO,IPTU,5438.40,10779.87,10779.87,0.00,...,1,1,0,1,0,4.0,3.0,15.0,2,0
8,000040eda866e3d19ceed47545e621dd2,mercantil,2f8ae1b9606267b4,INSTRUTOR DE TREINAMENTOS,INAPTO,ISS,0.00,328.63,328.63,0.00,...,0,1,0,0,0,1.0,0.0,15.0,0,0
10,0000521b64bf28c8e52281a70553db461,imovel,b5df816e1786f0a3,CASA,ATIVO,IPTU,0.00,490.83,245.50,245.33,...,1,1,0,0,0,10.0,0.0,7.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2414890,ffffd120a8f3f2799fcacff532cf50c41,imovel,934532c6c448a899,CASA,ATIVO,IPTU,0.00,802.85,617.95,184.90,...,1,1,0,0,0,16.0,0.0,9.0,0,0
2414891,ffffd79d8aa612ac9fcacff532cf50c41,imovel,c6b16a1235d987d3,APARTAMENTO,ATIVO,IPTU,26104.42,26104.42,22190.68,3913.74,...,1,1,1,0,0,40.0,40.0,1.0,0,0
2414893,ffffd8af5760cada9fcacff532cf50c41,imovel,d411f54ad97d1f39,NaN,ATIVO,IPTU,0.00,1728.13,790.85,937.28,...,0,0,0,0,0,19.0,0.0,4.0,0,0
2414894,ffffdb721b90282718565e26094f06ee2,mercantil,51d6043a4efa1ed1,PUBLICITARIO (NÍVEL MÉDIO),ATIVO,ISS,0.00,240.77,240.77,0.00,...,0,1,0,0,0,1.0,0.0,6.0,0,0


In [43]:
# Cdas que estão na query vw_da_predicao e não estao na query vw_da_treino
cdas_exclus = base_pred[~base_pred ['cda'].isin(base_conjunta['cda'])]
cdas_exclus

,cda,tipo_divida,id_contribuinte,atividade_principal,situacao,tipo_tributo,vlr_pago,valor_tot,vlr_tributo,vlr_taxa,...,edificacao,cpf_cnpj_existe,protesto,ajuizamento,refis,deb_totais,deb_pagos,idade_divida,quantidade_reparcelamento,da_aberto
59,0003ba44e5f5349018565e26094f06ee2,mercantil,7d81242e3b8c726b,REPRESENTANTES COMERCIAIS E AGENTES DO COM DE ...,INAPTO,ISS,0.0,19350.09,0.00,19350.09,...,0,1,0,1,0,10.0,0.0,32.0,0,1
116,0007e94628daa91c9989bae6f4af91ee2,mercantil,6b7cc348924a7465,LABORATÓRIOS FOTOGRÁFICOS,INAPTO,ISS,0.0,19676.10,0.00,19676.10,...,0,1,0,1,0,8.0,0.0,31.0,0,1
121,00080e7c0b550fe818565e26094f06ee1,imovel,3f9035f1eb8ccb43,NaN,ATIVO,IPTU,0.0,12648.40,5575.66,7072.74,...,0,1,0,1,0,9.0,0.0,29.0,0,1
156,000a7db0f19fc5c66b3a8e268c80aedf1,imovel,4dd5b3b1cc6b2111,CASA,ATIVO,IPTU,0.0,4944.76,0.00,4944.76,...,1,1,0,1,0,16.0,0.0,35.0,0,1
182,000c52e372558f8ae52281a70553db461,imovel,181362fbd68d0c2d,NaN,EM PROCESSO DE BAIXA,IPTU,0.0,19621.14,11544.49,8076.65,...,0,1,0,1,0,9.0,0.0,29.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985179,fff2e56a47e964879989bae6f4af91ee2,mercantil,a2ea00d6a346512e,COMÉRCIO VAREJISTA DE MATERIAIS DE CONSTRUÇÃO ...,SUSPENSO,ISS,0.0,9410.89,0.00,9410.89,...,0,1,0,1,0,9.0,0.0,29.0,0,1
985203,fff48f3dbc51bbd5e52281a70553db461,imovel,ef8f2604e3e671aa,CASA,ATIVO,IPTU,0.0,2915.14,2915.14,0.00,...,1,1,0,1,0,1.0,0.0,32.0,0,1
985313,fffa2eec8a837174e52281a70553db461,imovel,18a5d111b53fea86,NaN,EM PROCESSO DE BAIXA,IPTU,0.0,29049.34,16910.14,12139.20,...,0,0,0,1,0,12.0,0.0,29.0,0,1
985333,fffb75b9655228322b46f673bf0c61251,imovel,bfe526027cfcc6aa,APARTAMENTO,ATIVO,IPTU,0.0,4269.69,3431.23,838.46,...,1,1,0,1,0,2.0,0.0,29.0,0,1


In [66]:
aux = base_conjunta[base_conjunta['cda'] == 'ffffd79d8aa612ac9fcacff532cf50c42']
aux[['cda', 'tipo_divida', 'valor_tot', 'vlr_pago', 'id_contribuinte']]        

,cda,tipo_divida,valor_tot,vlr_pago,id_contribuinte
2414892,ffffd79d8aa612ac9fcacff532cf50c42,mercantil,2504.85,265.15,cc5afb41f6fbc367


In [65]:
aux2 = base_pred[base_pred['cda'] == 'ffffd79d8aa612ac9fcacff532cf50c42']
aux2[['cda', 'tipo_divida', 'valor_tot', 'vlr_pago']]

,cda,tipo_divida,valor_tot,vlr_pago
985396,ffffd79d8aa612ac9fcacff532cf50c42,mercantil,1073.08,0.0


In [33]:
duplicatas_conjunta = base_conjunta[base_conjunta['cda'].duplicated(keep=False)]
duplicatas_conjunta[['cda']]

,cda


In [19]:
soma_pred = base_pred['valor_tot'].sum()
soma_pred

14607029043.600006

In [5]:
base_conjunta.columns

Index(['cda', 'tipo_divida', 'id_contribuinte', 'atividade_principal',
       'situacao', 'tipo_tributo', 'vlr_pago', 'valor_tot', 'vlr_tributo',
       'vlr_taxa', 'competencia_divida', 'inscricao_divida',
       'arrecadacao_divida', 'ajuizamento_divida', 'edificacao',
       'cpf_cnpj_existe', 'protesto', 'ajuizamento', 'refis', 'deb_totais',
       'deb_pagos', 'idade_divida', 'quantidade_reparcelamento', 'da_aberto'],
      dtype='object')

---

In [6]:
print("Inicia transformação das variáveis sobre a dívida")
print("Processamento das variáveis sobre a dívida")

# Gera as variáveis de tempo
base_conjunta['data_divida'] = pd.to_datetime(base_conjunta['inscricao_divida'], infer_datetime_format = True)
base_conjunta['ano_inscricao_da'] = base_conjunta['data_divida'].dt.year

Inicia transformação das variáveis sobre a dívida
Processamento das variáveis sobre a dívida


In [7]:
# Seleciona dados sobre a divida
dados_divida = base_conjunta[
    ['cda', 'id_contribuinte', 'cpf_cnpj_existe',
     'da_aberto', 'atividade_principal', 'valor_tot','vlr_pago', 'vlr_tributo', 'vlr_taxa', 'arrecadacao_divida',
     'ajuizamento_divida', 'ajuizamento', 'protesto', 'competencia_divida', 'refis',
     'tipo_divida', 'ano_inscricao_da', 'quantidade_reparcelamento', 'situacao']]
dados_divida.dropna(subset=['id_contribuinte'], inplace=True)
dados_divida['id_contribuinte'] = dados_divida['id_contribuinte'].astype(str)  # persistindo tipo de dados

In [8]:
# Calcula a idade da dívida ativa
dados_divida['ano_atual'] = date.today().year
dados_divida['anos_idade_da'] = dados_divida['ano_atual'] - dados_divida['ano_inscricao_da']
dados_divida = dados_divida.drop(columns=['ano_atual'])

In [9]:
# Renomeia colunas para nome mais adequados e filtra dataframe
colunas_nome = {
    'valor_tot': 'valor_total_da',
}
df_divida_ativa = dados_divida.rename(columns=colunas_nome)

In [10]:
print("Processamento das variáveis sobre o contribuinte")

# Cria conexão com o banco e prepara os dados

def read_s3_files(bucket_name, folder_name, file_name):
    file_key_aws = folder_name + file_name
    obj = s3.Bucket(bucket_name).Object(file_key_aws).get()
    df = pd.read_csv(obj['Body'], sep=';')
    return df

dados_contribuinte = read_s3_files(bucket_name=os.getenv("S3_BUCKET_NAME"), folder_name=os.getenv("S3_FOLDER_NAME"),
                                   file_name='feature_store_contribuinte_2.csv')


Processamento das variáveis sobre o contribuinte


In [14]:
df_divida_ativa.columns

Index(['cda', 'id_contribuinte', 'cpf_cnpj_existe', 'da_aberto',
       'atividade_principal', 'valor_total_da', 'vlr_pago', 'vlr_tributo',
       'vlr_taxa', 'arrecadacao_divida', 'ajuizamento_divida', 'ajuizamento',
       'protesto', 'competencia_divida', 'refis', 'tipo_divida',
       'ano_inscricao_da', 'quantidade_reparcelamento', 'situacao',
       'anos_idade_da'],
      dtype='object')

In [15]:
dados_contribuinte.columns

Index(['id_pessoa', 'situacao', 'cpf_cnpj_existe', 'edificacao',
       'qtd_notas_2anos', 'situacao_ativa', 'status_situacao', 'deb_totais',
       'deb_pagos', 'valor_tot', 'vlr_pago', 'frequencia_da_pessoa',
       'total_debitos_pessoa', 'debitos_pagos_pessoa', 'valor_total_pessoa',
       'valor_pago_pessoa', 'historico_pagamento_em_qtd',
       'historico_pagamento_em_valor', 'class_contribuinte',
       'class_contribuinte_nome', 'class_contribuinte_peso'],
      dtype='object')

In [20]:
    contribuinte_aux =  dados_contribuinte[['id_pessoa', 'edificacao',
                                            'qtd_notas_2anos', 'situacao_ativa', 'status_situacao',
                                             'frequencia_da_pessoa', 'historico_pagamento_em_qtd',
                                             'historico_pagamento_em_valor', 'class_contribuinte',
                                            'class_contribuinte_nome', 'class_contribuinte_peso']]

In [ ]:
contribuinte_aux

In [24]:
# Realiza o merge dos dados da dívida com os dados do contribuinte

df = pd.merge(
    left=df_divida_ativa,
    right=contribuinte_aux,
    left_on='id_contribuinte',
    right_on='id_pessoa',
    how='left')

In [25]:
df.columns

Index(['cda', 'id_contribuinte', 'cpf_cnpj_existe', 'da_aberto',
       'atividade_principal', 'valor_total_da', 'vlr_pago', 'vlr_tributo',
       'vlr_taxa', 'arrecadacao_divida', 'ajuizamento_divida', 'ajuizamento',
       'protesto', 'competencia_divida', 'refis', 'tipo_divida',
       'ano_inscricao_da', 'quantidade_reparcelamento', 'situacao',
       'anos_idade_da', 'id_pessoa', 'edificacao', 'qtd_notas_2anos',
       'situacao_ativa', 'status_situacao', 'frequencia_da_pessoa',
       'historico_pagamento_em_qtd', 'historico_pagamento_em_valor',
       'class_contribuinte', 'class_contribuinte_nome',
       'class_contribuinte_peso'],
      dtype='object')

In [26]:
# # Ajustando pesos dos contribuintes, foi dado ao primeira dívida o mesmo peso do bom pagador, isso procede?
# df.loc[df['frequencia_da_pessoa'] <= 1, 'class_contribuinte_nome'] == 'PRIMEIRA DIVIDA'
# df.loc[df['class_contribuinte_nome'] == 'PRIMEIRA DIVIDA', 'class_contribuinte_peso'] == 2.54487

# df['class_contribuinte_nome'] = df['class_contribuinte_nome'].fillna('PIOR PAGADOR')
# df.loc[df['class_contribuinte_nome'] == 'PIOR PAGADOR', 'class_contribuinte_peso'] == -0.98031
# df.fillna(0, inplace=True)

# Início das previsões

In [27]:
print("Preparação do pipeline de previsões")
# Filtra os dados que precisamos para previsão
df_feature_store = df[['valor_total_da', 'anos_idade_da', #'quantidade_reparcelamentos',
                       'frequencia_da_pessoa', 'historico_pagamento_em_qtd', 'status_situacao',
                       'historico_pagamento_em_valor', 'class_contribuinte_peso']]

Preparação do pipeline de previsões


In [28]:
# Normaliza os dados para previsão utilizando o StandardScaler
normalizador = StandardScaler()
normalizador.fit(df_feature_store)
dados_normalizados = normalizador.fit_transform(df_feature_store)

colunas = list(normalizador.get_feature_names_out())
df_normalizado = pd.DataFrame(dados_normalizados, columns=colunas)

In [29]:
print("Carrega o modelo e realiza a previsão do Índice Geral de Recuperação (IGR)")


def abre_modelo(nome_modelo, path_modelo, zip_name=None):
    if zip_name:
        zip_file = os.path.join(path_modelo, zip_name)
        z = zipfile.ZipFile(zip_file)
        z.extract(nome_modelo)
    else:
        nome_modelo = os.path.join(path_modelo, nome_modelo)

    modelo = pickle.load(open(nome_modelo, 'rb'))
    return modelo


model_predict_igr = abre_modelo("modeloDA-igr-divida-v2.pkl", modelsPath)

Carrega o modelo e realiza a previsão do Índice Geral de Recuperação (IGR)


In [30]:
# Realizando previsões
previsoes = model_predict_igr.predict(df_normalizado)
df['igr'] = previsoes
df.loc[df['status_situacao'] == 0, 'igr'] = 0
df.loc[df['anos_idade_da'] >= 15, 'igr'] = 0

print("Início do processo de classificação do rating da dívida parametrizando junto a classificação do contribuinte")

def make_rating_divida(dataframe):
    dataframe.loc[dataframe['igr'] == 0, 'rating_divida'] = 'BAIXISSIMA'

    # Melhor Pagador
    dataframe.loc[(dataframe['rating_divida'].isnull()) & (dataframe['class_contribuinte'] == 2), 'rating_divida'] = 'ALTISSIMA'

    # Pior Pagador
    dataframe.loc[(dataframe['class_contribuinte'] == 0) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'

    # dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] >= 0.5), 'rating_divida'] = 'ALTISSIMA'
    # dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < 0.5) & (dataframe['igr'] >= 0.1), 'rating_divida'] = 'ALTA'
    # dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < 0.1) & (dataframe['igr'] != 0), 'rating_divida'] = 'MEDIA'

    # Pagador intermediario
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] >= 0.5), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] < 0.5) & (dataframe['igr'] >= 0.05), 'rating_divida'] = 'MEDIA'
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] < 0.1) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'

    # Bom pagador
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] >= 0.5), 'rating_divida'] = 'ALTISSIMA'
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < 0.5) & (dataframe['igr'] >= 0.05), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < 0.05) & (dataframe['igr'] != 0), 'rating_divida'] = 'MEDIA'

    # Melhor Pagador
    dataframe.loc[(dataframe['class_contribuinte'] == 4) & (dataframe['igr'] >= 0.3), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 4) & (dataframe['igr'] < 0.3) & dataframe['igr'] >= 0.1, 'rating_divida'] = 'MEDIA'
    dataframe.loc[(dataframe['class_contribuinte'] == 4) & (dataframe['igr'] < 0.1) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'
    return dataframe


df = make_rating_divida(df)

Início do processo de classificação do rating da dívida parametrizando junto a classificação do contribuinte


In [31]:
print("Inicia a conexão com S3 para inscrição dos dados com as previsões")

# Cria conexão ao s3 e preenche a tabela com os dados

def up_s3_files(dataframe, bucket_name, folder_name, file_name):
    csv_buffer = BytesIO()
    dataframe.to_csv(csv_buffer, sep=';', index=False)
    file_key_aws = folder_name + file_name
    s3.Object(bucket_name, file_key_aws).put(Body=csv_buffer.getvalue())


up_s3_files(dataframe=df,
            bucket_name=os.getenv("S3_BUCKET_NAME"),
            folder_name=os.getenv("S3_FOLDER_NAME"),
            file_name='baseline_beta.csv')

print("Upload de dados efetuados no s3")
print("Processo finalizado")
print("Arquivo disponível para download e análise")

Inicia a conexão com S3 para inscrição dos dados com as previsões
Upload de dados efetuados no s3
Processo finalizado
Arquivo disponível para download e análise
